In [7]:
# rag.ipynb

# --- IMPORTS ---
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter
import shutil
import gradio as gr

# --- LOAD ENV VARIABLES ---
load_dotenv(override=True)



# --- Initialize Gemini LLM ---
gemini_llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0.3)

try:
    _ = gemini_llm.invoke("Test LLM connection.")
    print("✅ Gemini LLM initialized and working.")
except Exception as e:
    print(f"❌ Error initializing Gemini LLM: {e}")
    exit()

# --- SETTINGS ---
DOCS_PATH = "knowledge_base"  # Folder with Markdown/Text files
CHROMA_DB_PATH = "chroma_db_fitflix"



✅ Gemini LLM initialized and working.


In [8]:
# --- STEP 1: Load Documents ---
loader = DirectoryLoader(DOCS_PATH, glob="**/*.md", loader_cls=TextLoader)
documents = loader.load()
print(f"✅ Loaded {len(documents)} documents.")

# --- STEP 2: Split Text into Chunks ---
text_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3")
    ]
)
docs = []
for doc in documents:
    docs.extend(text_splitter.split_text(doc.page_content))

print(f"✅ Split into {len(docs)} text chunks.")



✅ Loaded 9 documents.
✅ Split into 49 text chunks.


In [9]:
# --- STEP 3: Initialize Embeddings and Chroma Vector Store ---
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Remove old DB if exists
if os.path.exists(CHROMA_DB_PATH):
    shutil.rmtree(CHROMA_DB_PATH)

vectorstore = Chroma.from_texts([d.page_content for d in docs], embedding=embeddings, persist_directory=CHROMA_DB_PATH)
vectorstore.persist()
print("✅ Chroma Vector Store created.")



✅ Chroma Vector Store created.


C:\Users\dandu\AppData\Local\Temp\ipykernel_26456\3558046920.py:9: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [18]:
# --- STEP 4: Setup RAG Pipeline ---
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# Minimal response template for concise answers
prompt_template = """You are an AI assistant specialized in information about Fitflix entities.
Use the following pieces of context to answer the question at the end.
and try to interactive if any body greets you greet them also. and follow the humanity rules
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Keep the answer concise, professional, and directly address the question.


Context:
{context}

Question: {question}

Answer:
"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True,output_key="answer")

rag_chain = ConversationalRetrievalChain.from_llm(
    llm=gemini_llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": PROMPT},
    return_source_documents=True,
    verbose=False
)




In [19]:
# --- STEP 5: Create Gradio Chat Interface ---
def chat_fn(message, history):
    result = rag_chain.invoke({"question": message})
    answer = result["answer"]
    return answer

with gr.Blocks() as demo:
    gr.Markdown("## 🏋️ Fitflix RAG Chatbot (Powered by Gemini)")
    chatbot = gr.Chatbot(height=400)
    msg = gr.Textbox(label="Ask me about fitness, workouts, or diet...")
    clear = gr.Button("Clear Chat")

    def respond(message, chat_history):
        response = chat_fn(message, chat_history)
        chat_history.append((message, response))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch(share=False)


C:\Users\dandu\AppData\Local\Temp\ipykernel_26456\3466950162.py:9: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=400)


* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.
